In [ ]:
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip3 install -q "monai-weekly[nibabel, skimage, pillow, tensorboard, gdown, ignite, torchvision, itk, tqdm, lmdb, psutil,  openslide, pandas, einops, transformers, mlflow, matplotlib, tensorboardX, tifffile, imagecodecs]"
!pip3 install h5py
!python -c "import matplotlib" || pip install -q matplotlib
!pip install -q pytorch-lightning
%matplotlib inline

: 

In [2]:
import numpy as np

from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,

)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

import matplotlib.pyplot as plt


from monai.transforms import (
    LoadImage, LoadImaged, EnsureChannelFirstd,
    Resized,  Compose
)
from monai.config import print_config
import re
import pytorch_lightning
from monai.utils import set_determinism
from monai.transforms import (
    AsDiscrete,
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureTyped,
    EnsureType,
)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, list_data_collate, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
from monai.config import print_config
#print_config()


ModuleNotFoundError: No module named 'monai'

In [ ]:
set_determinism(seed=0)


In [ ]:
from monai.transforms import *

set_determinism(seed=0)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(keys=["image", "label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
    ]
)


In [ ]:
class Net(pytorch_lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=2,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH,
        )
        self.loss_function = DiceLoss(to_onehot_y=True, softmax=True)
        self.post_pred = Compose([EnsureType("tensor", device="cpu"), AsDiscrete(argmax=True, to_onehot=2)])
        self.post_label = Compose([EnsureType("tensor", device="cpu"), AsDiscrete(to_onehot=2)])
        self.dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
        self.best_val_dice = 0
        self.best_val_epoch = 0

    def forward(self, x):
        return self._model(x)

    def prepare_data(self):
        # set up the correct data path
        data_dir = "D:\\dataSets\\CTORGmini\\"

        train_images = sorted(glob.glob(os.path.join(data_dir, "volumes 0-49", "*.nii.gz")))

        train_labels = sorted( glob.glob(os.path.join(data_dir, "labels", "*.nii.gz")))

        data_dicts = [
            {"image": image_name, "label": label_name}
            for image_name, label_name in zip(train_images, train_labels)
        ]
        train_files, val_files = data_dicts[:-9], data_dicts[-9:]

        # set deterministic training for reproducibility
        set_determinism(seed=0)

        # define the data transforms
        train_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                AddChanneld(keys=["image", "label"]),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    pixdim=(1.5, 1.5, 2.0),
                    mode=("bilinear", "nearest"),
                ),
                ScaleIntensityRanged(
                    keys=["image"], a_min=-57, a_max=164,
                    b_min=0.0, b_max=1.0, clip=True,
                ),
                CropForegroundd(keys=["image", "label"], source_key="image"),
                # randomly crop out patch samples from
                # big image based on pos / neg ratio
                # the image centers of negative samples
                # must be in valid image area
                RandCropByPosNegLabeld(
                    keys=["image", "label"],
                    label_key="label",
                    spatial_size=(96, 96, 96),
                    pos=1,
                    neg=1,
                    num_samples=4,
                    image_key="image",
                    image_threshold=0,
                ),
                # user can also add other random transforms
                #                 RandAffined(
                #                     keys=['image', 'label'],
                #                     mode=('bilinear', 'nearest'),
                #                     prob=1.0,
                #                     spatial_size=(96, 96, 96),
                #                     rotate_range=(0, 0, np.pi/15),
                #                     scale_range=(0.1, 0.1, 0.1)),
                EnsureTyped(keys=["image", "label"]),
            ]
        )
        val_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                AddChanneld(keys=["image", "label"]),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    pixdim=(1.5, 1.5, 2.0),
                    mode=("bilinear", "nearest"),
                ),
                ScaleIntensityRanged(
                    keys=["image"], a_min=-57, a_max=164,
                    b_min=0.0, b_max=1.0, clip=True,
                ),
                CropForegroundd(keys=["image", "label"], source_key="image"),
                EnsureTyped(keys=["image", "label"]),
            ]
        )

        # we use cached datasets - these are 10x faster than regular datasets
        self.train_ds = CacheDataset(
            data=train_files, transform=train_transforms,
            cache_rate=1.0, num_workers=4,
        )
        self.val_ds = CacheDataset(
            data=val_files, transform=val_transforms,
            cache_rate=1.0, num_workers=4,
        )
#         self.train_ds = monai.data.Dataset(
#             data=train_files, transform=train_transforms)
#         self.val_ds = monai.data.Dataset(
#             data=val_files, transform=val_transforms)

    def train_dataloader(self):
        train_loader = torch.utils.data.DataLoader(
            self.train_ds, batch_size=2, shuffle=True,
            num_workers=4, collate_fn=list_data_collate,
        )
        return train_loader

    def val_dataloader(self):
        val_loader = torch.utils.data.DataLoader(
            self.val_ds, batch_size=1, num_workers=4)
        return val_loader

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self._model.parameters(), 1e-4)
        return optimizer

    def training_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        output = self.forward(images)
        loss = self.loss_function(output, labels)
        tensorboard_logs = {"train_loss": loss.item()}
        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        outputs = sliding_window_inference(
            images, roi_size, sw_batch_size, self.forward)
        loss = self.loss_function(outputs, labels)
        outputs = [self.post_pred(i) for i in decollate_batch(outputs)]
        labels = [self.post_label(i) for i in decollate_batch(labels)]
        self.dice_metric(y_pred=outputs, y=labels)
        return {"val_loss": loss, "val_number": len(outputs)}

    def validation_epoch_end(self, outputs):
        val_loss, num_items = 0, 0
        for output in outputs:
            val_loss += output["val_loss"].sum().item()
            num_items += output["val_number"]
        mean_val_dice = self.dice_metric.aggregate().item()
        self.dice_metric.reset()
        mean_val_loss = torch.tensor(val_loss / num_items)
        tensorboard_logs = {
            "val_dice": mean_val_dice,
            "val_loss": mean_val_loss,
        }
        if mean_val_dice > self.best_val_dice:
            self.best_val_dice = mean_val_dice
            self.best_val_epoch = self.current_epoch
        print(
            f"current epoch: {self.current_epoch} "
            f"current mean dice: {mean_val_dice:.4f}"
            f"\nbest mean dice: {self.best_val_dice:.4f} "
            f"at epoch: {self.best_val_epoch}"
        )
        return {"log": tensorboard_logs}

In [7]:
data_dir = "D:\\dataSets\\CTORGmini\\"

train_images = sorted(
    glob.glob(os.path.join(data_dir, "volumes 0-49", "*.nii.gz")))

train_labels = sorted(
    glob.glob(os.path.join(data_dir, "labels", "*.nii.gz")))

data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]
data_dicts

[{'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-0.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-0.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-1.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-1.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-2.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-2.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-3.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-3.nii.gz'}]

In [8]:
def getNumberFromFilePath(dat):
    string = str(dat['image_meta_dict']['filename_or_obj'])
    numb = re.search('.nii', string).span()[0] ## this does what you wanted
    describingNumber = string[(numb-2):(numb)].replace("-","")
    return int(describingNumber)

In [9]:
# https://github.com/Project-MONAI/MONAI/discussions/3906
# below example how to filter out the files without spacing and orientation in metadata 
# >>> from monai.transforms import LoadImaged
# >>> data_dict = LoadImaged(keys="img")({"img": "./volume-20.nii.gz"})
# >>> (data_dict["img_meta_dict"]["qform_code"], data_dict["img_meta_dict"]["sform_code"])
# (array(0, dtype=int16), array(0, dtype=int16))

# >>> data_dict = LoadImaged(keys="img")({"img": "./volume-21.nii.gz"})
# >>> (data_dict["img_meta_dict"]["qform_code"], data_dict["img_meta_dict"]["sform_code"])
# (array(1, dtype=int16), array(1, dtype=int16))


In [10]:
check_ds = Dataset(data=data_dicts, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)

for dat in check_loader:
    # describingNumber = getNumberFromFilePath(dat)
    # grp = f.create_group(str(describingNumber))

    # labelBoolTensor =torch.where( dat['label']==1, 1, 0).bool()
    # summ= torch.sum(labelBoolTensor)
    # #if summ>0:
    # labelBoolDataset = grp.create_dataset('liver', data=  (labelBoolTensor[0,0,:,:,:]).detach().numpy())
    # imageDataset = grp.create_dataset('image', data=  (dat['image'][0,0,:,:,:]).detach().numpy())
    # labelBoolDataset.attrs['sum']=  summ
    
    # print( labelBoolTensor[0,0,:,:,:].size() )

    # imageDataset.attrs['dataType']=  "CT"
    # labelBoolDataset.attrs['dataType']=  "boolLabel"

    # #seting spacing - needs to be the same for all of the arrays in a grop - so the same for all images in patient
    # grp.attrs['spacing']=(1,1,1)

    # #get minimum and maximum values
    # imageDataset.attrs['min']=torch.min(dat['image'])
    # imageDataset.attrs['max']=torch.max(dat['image'])

    # labelBoolDataset.attrs['min']=0
    # labelBoolDataset.attrs['max']=1

  

    # print(describingNumber)
    pass